In [1]:
import os
import sys
import glob
import math
import pandas as pd
import numpy as np
import itertools
import random
import requests
import xml.etree.ElementTree as ET
import csv

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

#import tensorflow_probability as tfp
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import EarlyStopping
from IPython.display import SVG

from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, normalize
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')

from ipywidgets import FloatProgress
from IPython.display import display

fig_width = 12
plt.rcParams["font.size"] = 40
plt.rcParams['axes.labelsize'] = 40
plt.rcParams['axes.labelweight'] = 'bold'
#plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
#plt.rcParams.keys()

from codes.utils.load_utils import num_labels, time_change, setup_dir, find_xml_filenames, find_csv_filenames
from extendedMD import emd

>>> Done:
>>> Done: Succesfully imported Utils module


In [2]:
#data = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\labeled\LB01_l_acc.csv")
data = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\l_2022_acc.csv")

In [3]:
data

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val,year
0,LB01,1661544781000,0.011719,-0.105469,0.273438,unlabeled,0,2022
1,LB01,1661544781032,0.105469,-0.199219,0.574219,unlabeled,0,2022
2,LB01,1661544781064,-0.183594,0.175781,0.066406,unlabeled,0,2022
3,LB01,1661544781096,-0.550781,-0.027344,-0.144531,unlabeled,0,2022
4,LB01,1661544781129,-0.363281,-0.132813,-0.035156,unlabeled,0,2022
...,...,...,...,...,...,...,...,...
22942599,LB17,1663492499838,-0.062500,-0.234375,2.093750,unlabeled,0,2022
22942600,LB17,1663492499870,-0.433594,-0.074219,2.031250,unlabeled,0,2022
22942601,LB17,1663492499903,-0.304688,-0.320313,1.878906,unlabeled,0,2022
22942602,LB17,1663492499935,-0.062500,-0.554688,1.621094,unlabeled,0,2022


In [4]:
data["timestamp"] = pd.to_datetime(data["timestamp"],unit='ms')

In [5]:
bird_list = list(data.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
bird_list

['LB01',
 'LB02',
 'LB03',
 'LB04',
 'LB05',
 'LB07',
 'LB08',
 'LB09',
 'LB10',
 'LB11',
 'LB13',
 'LB14',
 'LB15',
 'LB17']

In [6]:
data = data[data['animal_tag']==bird_list[0]]
data

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val,year
0,LB01,2022-08-26 20:13:01.000,0.011719,-0.105469,0.273438,unlabeled,0,2022
1,LB01,2022-08-26 20:13:01.032,0.105469,-0.199219,0.574219,unlabeled,0,2022
2,LB01,2022-08-26 20:13:01.064,-0.183594,0.175781,0.066406,unlabeled,0,2022
3,LB01,2022-08-26 20:13:01.096,-0.550781,-0.027344,-0.144531,unlabeled,0,2022
4,LB01,2022-08-26 20:13:01.129,-0.363281,-0.132813,-0.035156,unlabeled,0,2022
...,...,...,...,...,...,...,...,...
2446484,LB01,2022-08-28 05:06:20.838,-0.062500,-0.093750,1.839844,unlabeled,0,2022
2446485,LB01,2022-08-28 05:06:20.870,-0.066406,-0.066406,1.863281,unlabeled,0,2022
2446486,LB01,2022-08-28 05:06:20.903,-0.105469,-0.277344,1.867188,unlabeled,0,2022
2446487,LB01,2022-08-28 05:06:20.935,-0.156250,-0.507813,1.882813,unlabeled,0,2022


In [7]:
data['date'] =data['timestamp'].dt.strftime('%Y-%m-%d')

In [8]:
data

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val,year,date
0,LB01,2022-08-26 20:13:01.000,0.011719,-0.105469,0.273438,unlabeled,0,2022,2022-08-26
1,LB01,2022-08-26 20:13:01.032,0.105469,-0.199219,0.574219,unlabeled,0,2022,2022-08-26
2,LB01,2022-08-26 20:13:01.064,-0.183594,0.175781,0.066406,unlabeled,0,2022,2022-08-26
3,LB01,2022-08-26 20:13:01.096,-0.550781,-0.027344,-0.144531,unlabeled,0,2022,2022-08-26
4,LB01,2022-08-26 20:13:01.129,-0.363281,-0.132813,-0.035156,unlabeled,0,2022,2022-08-26
...,...,...,...,...,...,...,...,...,...
2446484,LB01,2022-08-28 05:06:20.838,-0.062500,-0.093750,1.839844,unlabeled,0,2022,2022-08-28
2446485,LB01,2022-08-28 05:06:20.870,-0.066406,-0.066406,1.863281,unlabeled,0,2022,2022-08-28
2446486,LB01,2022-08-28 05:06:20.903,-0.105469,-0.277344,1.867188,unlabeled,0,2022,2022-08-28
2446487,LB01,2022-08-28 05:06:20.935,-0.156250,-0.507813,1.882813,unlabeled,0,2022,2022-08-28


In [9]:
data26 = data[data['date']=='2022-08-26']
data27 = data[data['date']=='2022-08-27']
data28 = data[data['date']=='2022-08-28']

In [10]:
data26

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val,year,date
0,LB01,2022-08-26 20:13:01.000,0.011719,-0.105469,0.273438,unlabeled,0,2022,2022-08-26
1,LB01,2022-08-26 20:13:01.032,0.105469,-0.199219,0.574219,unlabeled,0,2022,2022-08-26
2,LB01,2022-08-26 20:13:01.064,-0.183594,0.175781,0.066406,unlabeled,0,2022,2022-08-26
3,LB01,2022-08-26 20:13:01.096,-0.550781,-0.027344,-0.144531,unlabeled,0,2022,2022-08-26
4,LB01,2022-08-26 20:13:01.129,-0.363281,-0.132813,-0.035156,unlabeled,0,2022,2022-08-26
...,...,...,...,...,...,...,...,...,...
422184,LB01,2022-08-26 23:59:59.838,0.031250,0.035156,0.953125,unlabeled,0,2022,2022-08-26
422185,LB01,2022-08-26 23:59:59.870,0.027344,0.031250,0.929688,unlabeled,0,2022,2022-08-26
422186,LB01,2022-08-26 23:59:59.903,0.027344,0.015625,0.906250,unlabeled,0,2022,2022-08-26
422187,LB01,2022-08-26 23:59:59.935,0.019531,-0.003906,0.886719,unlabeled,0,2022,2022-08-26


In [38]:
data["timestamp"] = pd.to_datetime(data["timestamp"],unit='ms')
bird_list = list(data.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
data = data[data['animal_tag']==bird_list[0]]
data['date'] =data['timestamp'].dt.strftime('%Y-%m-%d')
date_list = list(data.drop_duplicates(subset=['date'],keep = 'first')['date'])
data = 

data = data[data['animal_tag']==bird_list[0]]
data['date'] =data['timestamp'].dt.strftime('%Y-%m-%d')
day_data_df = data[data['date']=='2022-08-26']
iterations = int(len(day_data_df)/200000)
day_data_df_l = []
day_data_df

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val,year,date
0,LB01,2022-08-26 20:13:01.000,0.011719,-0.105469,0.273438,unlabeled,0,2022,2022-08-26
1,LB01,2022-08-26 20:13:01.032,0.105469,-0.199219,0.574219,unlabeled,0,2022,2022-08-26
2,LB01,2022-08-26 20:13:01.064,-0.183594,0.175781,0.066406,unlabeled,0,2022,2022-08-26
3,LB01,2022-08-26 20:13:01.096,-0.550781,-0.027344,-0.144531,unlabeled,0,2022,2022-08-26
4,LB01,2022-08-26 20:13:01.129,-0.363281,-0.132813,-0.035156,unlabeled,0,2022,2022-08-26
...,...,...,...,...,...,...,...,...,...
422184,LB01,2022-08-26 23:59:59.838,0.031250,0.035156,0.953125,unlabeled,0,2022,2022-08-26
422185,LB01,2022-08-26 23:59:59.870,0.027344,0.031250,0.929688,unlabeled,0,2022,2022-08-26
422186,LB01,2022-08-26 23:59:59.903,0.027344,0.015625,0.906250,unlabeled,0,2022,2022-08-26
422187,LB01,2022-08-26 23:59:59.935,0.019531,-0.003906,0.886719,unlabeled,0,2022,2022-08-26


In [ ]:
data["timestamp"] = pd.to_datetime(data["timestamp"],unit='ms')
bird_list = list(data.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
for bird in bird_list:
    data = data[data['animal_tag']==bird_list[0]]
    data['date'] =data['timestamp'].dt.strftime('%Y-%m-%d')
    date_list = list(data.drop_duplicates(subset=['date'],keep = 'first')['date'])
    for date in date_list:
        day_data_df = data[data['date']==date]
        df_size = 200000
        day_data_df_l = []
        iterations = int(len(day_data_df)/df_size)
        initial_p=0
        for it in range(iterations+1):
            if it == iterations:
                exdata_df = day_data_df.iloc[initial_p:len(day_data_df)]
            else:
                exdata_df = day_data_df.iloc[initial_p:(initial_p+df_size)]            
            

            new_data = exdata_df.drop(["timestamp","label","date","year","l_val","animal_tag"],axis=1)
            #motif_candidates_dic_list, ts_1d = emd.find_motifs_from_emd(new_data, 10, 30, 10, 10, adaptive_break_points=True, z_threshold=0.01)
            print(new_data)
            day_data_df_l.append(exdata_df)
            initial_p = initial_p+df_size
            print(initial_p)       

In [39]:
date_list = list(data.drop_duplicates(subset=['date'],keep = 'first')['date'])

In [40]:
date_list

['2022-08-26', '2022-08-27', '2022-08-28']

In [35]:
for it in iterations:
    

7

In [11]:
data26.drop(["timestamp","label","date","year","l_val","animal_tag"],axis=1,inplace=True)
data27.drop(["timestamp","label","date","year","l_val","animal_tag"],axis=1,inplace=True)
data28.drop(["timestamp","label","date","year","l_val","animal_tag"],axis=1,inplace=True)

In [12]:
new_data = data26.iloc[0:200000]

In [13]:
new_data

,acc_x,acc_y,acc_z
0,0.011719,-0.105469,0.273438
1,0.105469,-0.199219,0.574219
2,-0.183594,0.175781,0.066406
3,-0.550781,-0.027344,-0.144531
4,-0.363281,-0.132813,-0.035156
...,...,...,...
199995,0.074219,0.058594,1.019531
199996,0.074219,0.054688,1.007813
199997,0.066406,0.046875,0.992188
199998,0.062500,0.054688,0.972656


In [14]:
motif_candidates_dic_list, ts_1d = emd.find_motifs_from_emd(new_data, 10, 30, 10, 10, adaptive_break_points=True, z_threshold=0.01)

Motif candidates of size 1 successfully extracted
Motif candidates of size 2 successfully extracted
Motif candidates of size 3 successfully extracted
Motif candidates of size 4 successfully extracted
Motif candidates of size 5 successfully extracted
Motif candidates of size 6 successfully extracted
Motif candidates of size 7 successfully extracted
Motif candidates of size 8 successfully extracted
Motif candidates of size 9 successfully extracted
Motif candidates of size 10 successfully extracted
Motif candidates of size 11 successfully extracted
Motif candidates of size 12 successfully extracted
Motif candidates of size 13 successfully extracted
Motif candidates of size 14 successfully extracted
Motif candidates of size 15 successfully extracted
Motif candidates of size 16 successfully extracted
Motif candidates of size 17 successfully extracted
Motif candidates of size 18 successfully extracted
Motif candidates of size 19 successfully extracted
Motif candidates of size 20 successfully

In [15]:
len(ts_1d)

200000

In [16]:
len(motif_candidates_dic_list[1]['members_ts_pointers'][0])

30

In [17]:
motif_candidates_dic_list[2]

{'pattern': ['igcabcgiie'],
 'mdl_cost': 3321115.71,
 'mean_dist': 0.38,
 'members_ts_pointers': [[154507,
   154508,
   154509,
   154510,
   154511,
   154512,
   154513,
   154514,
   154515,
   154516,
   154517,
   154518,
   154519,
   154520,
   154521,
   154522,
   154523,
   154524,
   154525,
   154526,
   154527,
   154528,
   154529,
   154530,
   154531,
   154532,
   154533,
   154534,
   154535,
   154536],
  [175247,
   175248,
   175249,
   175250,
   175251,
   175252,
   175253,
   175254,
   175255,
   175256,
   175257,
   175258,
   175259,
   175260,
   175261,
   175262,
   175263,
   175264,
   175265,
   175266,
   175267,
   175268,
   175269,
   175270,
   175271,
   175272,
   175273,
   175274,
   175275,
   175276]],
 'center_ts_pointers': [154507,
  154508,
  154509,
  154510,
  154511,
  154512,
  154513,
  154514,
  154515,
  154516,
  154517,
  154518,
  154519,
  154520,
  154521,
  154522,
  154523,
  154524,
  154525,
  154526,
  154527,
  154528,

In [50]:
len(motif_candidates_dic_list[6]['center_ts_pointers'])

30

In [43]:
len(motif_candidates_dic_list[0]['members_ts_pointers'])

2

In [26]:
len(ts_1d)

200000

In [27]:
data26_ts_df = pd.DataFrame({'PCA_ts':ts_1d})
data26_ts_df

,PCA_ts
0,-0.162757
1,-0.007031
2,-0.415162
3,-0.187775
4,-0.120543
...,...
199995,-0.065609
199996,-0.066305
199997,-0.062321
199998,-0.074676


In [52]:
for cand in motif_candidates_dic_list:
    print(cand['pattern'])
    m_df_fn = cand['pattern']
    m_mdl_cost = [cand['mdl_cost']]*len(cand['center_ts_pointers'])
    m_mean_dist = [cand['mean_dist']]*len(cand['center_ts_pointers'])
    m_df = pd.DataFrame({'mdl_cost':m_mdl_cost, 'mean_dist':m_mean_dist,'center_ts_pointers':cand['center_ts_pointers']})

['jiebabefgi']
['hijigdbbbb']
['igcabcgiie']
['hiihdaabfh']
['aabhihgggg']
['acfhiiibdg']
['hjjhebbbcc']
['fijihfdbab']
['iiiiecbbbd']
['jjhhdcbbbc']
['iihdbabchi']
['gebabeghij']
['hgggiifbaa']
['bdijicadff']
['iihfbaadhh']
['fhiihecabh']
['hiiihebbbb']
['efecabfiji']
['hhiiifcaac']
['gedbbbchjj']
['bbefijidbd']
['bchjiigdba']
['bbcbdgijig']
['ifefhjgcaa']
['ccbbbeiiij']
['fcbbcdfijj']
['giiigfedaa']
['iiiigdbabd']
['iijfcccccc']
['hjigbaacgg']
['aacfffhiih']
['jjigddcbbc']
['jihihcbabe']
['iebabfiihe']
['bbabeiihhi']
['cbbbdhjiif']
['hiiihfcbaa']
['ghijhdaabe']
['bbabfijihf']
['cdcabgiiih']
['jigfcabcfh']
['iigcaadhhg']
['ijieaadeff']
['dcbchjjhda']
['ijihfdbbbb']
['ijigedcbbb']
['acfjjheccd']
['abhijifcce']
['chijiebabf']
['ecaacgiiii']
['jjgecccccd']
['aadfdgiihh']
['hcbabcgiii']
['djjddddddd']
['ijidbbbdef']
['hdbbbcfiji']
['ijjfdcbcbc']
['ijifbabcge']
['ccbbddhiji']
['jifecbabhh']
['cbbbdfgijj']
['jigfdbbcde']
['gfijigbabc']
['fccbbcfiji']
['ijigdbbadf']
['hiiihdbbbb']
['jihdbbdd